In [4]:
from cassandra.cluster import Cluster 

In [38]:
cluster = Cluster(["172.17.0.2"])
session = cluster.connect()

In [39]:
session.set_keyspace('ex1')

In [61]:
result_set = session.execute("select * from ex1.scores")
for result in result_set:
    print(result)

Row(user='pcmanus', game='Coup', year=2015, month=5, day=1, score=4000)
Row(user='pcmanus', game='Coup', year=2015, month=6, day=2, score=2000)
Row(user='pcmanus', game='Coup', year=2019, month=6, day=16, score=1000)
Row(user='yukim', game='Coup', year=2015, month=5, day=3, score=2250)
Row(user='jmckenzie', game='Coup', year=2015, month=6, day=1, score=2000)
Row(user='iamaleksey', game='Coup', year=2015, month=6, day=1, score=2500)
Row(user='tjake', game='Coup', year=2015, month=5, day=3, score=500)
Row(user='tjake', game='Coup', year=2015, month=6, day=2, score=1000)
Row(user='jbellis', game='Coup', year=2015, month=5, day=3, score=1750)
Row(user='cfranken', game='Coup', year=2017, month=2, day=3, score=20)
Row(user='jmanor', game='Coup', year=2018, month=5, day=3, score=20)


In [22]:
result_set = session.execute("select user, game, score from ex1.scores")
for result in result_set:
    print(result.game, result.score)

Coup 4000
Coup 2000
Coup 2250
Coup 2000
Coup 2500
Coup 500
Coup 1000
Coup 1750
Coup 20
Coup 20


In [29]:
user = 'pcmanus'

result_set = session.execute("""
    select user, game, score 
    from ex1.scores
    where user='{}'""".format(user)
)
for result in result_set:
    print(result.game, result.score)

Coup 4000
Coup 2000


In [37]:
user = "pcmanus"
game = "Coup"

result_set = session.execute("""
    select user, game, score 
    from ex1.scores
    where user=%(user)s and game=%(game)s""",
    {"user": user, "game": game}
)
for result in result_set:
    print(result.game, result.score)

Coup 4000
Coup 2000


In [47]:
#user = "pcmanus"
#game = "Coup"
arguments = {
    "gracz": "pcmanus",
    "gra": "Coup"
}

result_set = session.execute("""
    select user, game, score 
    from ex1.scores
    where user=%(gracz)s and game=%(gra)s""",
    # {"user": user, "game": game}
    arguments
)
for result in result_set:
    print(result.game, result.score)

Coup 4000
Coup 2000


In [54]:
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

In [55]:
session.set_keyspace('ex1')

In [59]:
#session.defaul_consistency_level = ConsistencyLevel.TWO
stmt = SimpleStatement("""
    INSERT INTO scores(user,game,year,month,day,score)
    VALUES
    ('pcmanus','Coup',2019, 6, 16, %s)""",
    consistency_level=ConsistencyLevel.QUORUM
)
session.execute(stmt, [1000])

In [60]:
#session.defaul_consistency_level = ConsistencyLevel.TWO
stmt = SimpleStatement("""
    INSERT INTO scores(user,game,year,month,day,score)
    VALUES
    ('pcmanus','Coup',2019, 6, 16, %s)
    IF NOT EXISTS""",
    consistency_level=ConsistencyLevel.QUORUM
)
session.execute(stmt, [1000])


In [87]:
from time import time

start = time()
result_set = session.execute("""
    select user, game, score from ex1.scores where user ='pcmanus'""")
took = time() - start
print ("TOOK", took * 1000)

for result in result_set:
    print(result.game, result.score)

TOOK 4.7149658203125
Coup 4000
Coup 2000
Coup 1000


In [98]:
from time import time

start = time()
future = session.execute_async("""
    select user, game, score from ex1.scores where user ='pcmanus'""")
took = time() - start
print ("TOOK", took * 1000)

result_set = future.result()
for result in result_set:
    print(result.game, result.score)

TOOK 1.4050006866455078
Coup 4000
Coup 2000
Coup 1000
